Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),<br>
("2023-11-21", "Electronics", 110, 13000),<br>
("2023-11-22", "Electronics", 105, 12500),<br>
("2023-11-20", "Clothing", 300, 15000),<br>
("2023-11-21", "Clothing", 280, 14000),<br>
("2023-11-22", "Clothing", 320, 16000),<br>
("2023-11-20", "Books", 150, 9000),<br>
("2023-11-21", "Books", 200, 12000),<br>
("2023-11-22", "Books", 180, 10000)<br>

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql.types import DoubleType

In [ ]:
# Создание Spark сессии
spark = SparkSession.builder.appName("Homework4").getOrCreate()

# Берем исходные данные
data = [("2023-11-20", "Electronics", 100, 12000),
        ("2023-11-21", "Electronics", 110, 13000),
        ("2023-11-22", "Electronics", 105, 12500),
        ("2023-11-20", "Clothing", 300, 15000),
        ("2023-11-21", "Clothing", 280, 14000),
        ("2023-11-22", "Clothing", 320, 16000),
        ("2023-11-20", "Books", 150, 9000),
        ("2023-11-21", "Books", 200, 12000),
        ("2023-11-22", "Books", 180, 10000)]

In [ ]:
# Создаем DataFrame с наименование столбцов
head = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, head)



In [ ]:
# Преобразуем поле date в тип данных DateType для работы с датами
df = df.withColumn("date", fn.to_date("date"))



In [ ]:
# Определяем оконную партицию
window_part = Window.partitionBy("category")



In [ ]:
# Оконная функция: вычисление средней выручки от продаж для каждой категории продукта
avg_revenue = fn.avg("revenue").over(window_part)



In [ ]:
# Применение оконной функции и приведение к double
result_df = df.withColumn("avg_revenue", avg_revenue.cast(DoubleType()))



In [ ]:
# исключаем повторы, используя функцию  distinct()
result_df.select(["category", "avg_revenue"]).distinct().show()

+-----------+------------------+
|   category|       avg_revenue|
+-----------+------------------+
|      Books|10333.333333333334|
|   Clothing|           15000.0|
|Electronics|           12500.0|
+-----------+------------------+



In [ ]:
# С помощью функции pivot перевернем значения ( из строки в столбец). Найдем средние значения выручки от продаж по категориям по датам
pivot_df = df.groupBy("category").pivot("date").agg(fn.avg("revenue")).orderBy("category")

# Вывод результатов
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+

